In [3]:
from bokeh.plotting import figure, show, output_file
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral6
from bokeh.transform import linear_cmap

import math
import pandas as pd
import numpy as np


output_notebook()

Loading BokehJS ...

In [4]:
full_path = 'Japan storms all years.csv'
all_info = pd.read_csv(full_path, sep=';')
tracks = all_info[all_info['Time of analysis'] != 66666]
#tracks[tracks['Landfall']=='#'].tail(200)
#tracks[tracks['Name']=='JEBI_1821'].tail(200)
tracks
all_info

,Name,Time of analysis,Indicator,Grade,Lat,Lon,Central Pressure,Max sustained wind speed,a,b,c,d,Landfall
0,19901017_5101,51021906,2,2,200,1385,1010,NaN,NaN,NaN,NaN,NaN,NaN
1,19901017_5101,51021912,2,2,200,1385,1010,NaN,NaN,NaN,NaN,NaN,NaN
2,19901017_5101,51021918,2,2,230,1421,1000,NaN,NaN,NaN,NaN,NaN,NaN
3,19901017_5101,51022000,2,9,250,1460,994,NaN,NaN,NaN,NaN,NaN,NaN
4,19901017_5101,51022006,2,9,276,1506,994,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65203,USAGI_1829,18112518,2,3,113,1066,1004,35.0,0.0,0.0,90090.0,90.0,NaN
65204,USAGI_1829,18112600,2,2,114,1059,1008,0.0,NaN,NaN,NaN,NaN,NaN
65205,USAGI_1829,18112606,2,2,115,1054,1008,0.0,NaN,NaN,NaN,NaN,NaN
65206,USAGI_1829,18112612,2,2,117,1049,1010,0.0,NaN,NaN,NaN,NaN,NaN


In [5]:
def merc(lat, lon, unit=0.1):
    lat =lat * 0.1
    lon = lon * 0.1
    r_major = 6378137.000
    x = r_major * np.deg2rad(lon)
    scale = x/lon
    y = 180.0/math.pi * np.log(np.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale
    return (x, y)

scaled_lat,scaled_lon = merc(tracks.Lat.values, tracks.Lon.values)
tracks['scaled_lat'] = scaled_lat
tracks['scaled_lon'] = scaled_lon
tracks['Max sustained wind speed km_per_h'] = tracks['Max sustained wind speed'] * 1.852

In [6]:
TOOLTIPS = [
    ("Name", "@name"),
   # ("(lat,lon)", "(@lat, @lon)"),
    ("Central Pressure", "@pressure"),
    ("Category", "@category"),
    #("Max sustained wind speed in knot", "@max_wind_speed")
    ("Max sustained wind speed in km/h", "@max_wind_speed")
]
p = figure(x_range=(14000000, 16000000), y_range=(1500000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator",tools='pan,hover,wheel_zoom',tooltips=TOOLTIPS)
p.add_tile(CARTODBPOSITRON)

TileRenderer(id='1039', ...)

In [7]:
mapper = linear_cmap(field_name='category', palette=Spectral6, 
                         low=min(tracks['Grade'].values),
                         high=max(tracks['Grade'].values))

In [8]:
paths = tracks.groupby('Name')
print(paths.groups.keys())
#print(paths.get_group('JEBI'))

dict_keys(['19890601_5108', '19890601_5110', '19890601_5311', '19890601_5314', '19890601_5315', '19890601_5321', '19890601_5322', '19890601_5401', '19890601_5407', '19890601_5409', '19890601_5411', '19890601_5418', '19890601_5423', '19890601_5502', '19890601_5506', '19890601_5512', '19890601_5515', '19890601_5516', '19890601_5517', '19890601_5518', '19890601_5524', '19890601_5602', '19890601_5604', '19890601_5608', '19890601_5616', '19890601_5701', '19890601_5708', '19890601_5709', '19890601_5718', '19890601_5802', '19890601_5804', '19890601_5806', '19890601_5814', '19890601_5816', '19890601_5818', '19890601_5820', '19890601_5824', '19890601_5831', '19890601_6010', '19890601_6019', '19890601_6106', '19890601_6112', '19890601_6114', '19890601_6219', '19890601_6611', '19890601_6620', '19890601_6625', '19890601_6632', '19890601_6715', '19890601_6731', '19890601_7006', '19890601_7124', '19890601_7133', '19890601_7211', '19890601_7521', '19901017_5101', '19901017_5303', '19910716_5107', '19

In [9]:
#storms_to_plot = paths.groups.keys()
#storms_to_plot = ['JEBI_1821', 'TRAMI_1824', 'VERA_5915','SONGDA_418','MIREILLE_9119','NANCY_6118','JELAWAT_1217']
storms_to_plot = ['JEBI_1821', 'TRAMI_1824','SONGDA_418','JELAWAT_1217']

In [10]:
for storm in storms_to_plot:
    path = paths.get_group(storm)
    source = ColumnDataSource(
        data=dict(lon=path.scaled_lon.values, lat=path.scaled_lat.values, pressure=path['Central Pressure'].values,
                  name=path.Name.values, category=path.Grade.values, max_wind_speed=path['Max sustained wind speed km_per_h'].values)
    )

    p.line(x="lat", y="lon", line_color="blue", line_alpha=0.8, source=source)
    p.circle(x="lat", y="lon", size=7, fill_color=mapper,color=mapper, fill_alpha=0.8, source=source)
    #fill_color="blue", size="category"
show(p)

In [ ]:
1